In [1]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
import pandas as pd
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input
import tensorflow as tf

2024-10-27 14:23:22.715207: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-27 14:23:22.858012: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-27 14:23:22.923265: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-27 14:23:22.943142: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-27 14:23:23.050770: I tensorflow/core/platform/cpu_feature_guar

In [28]:
tf.__version__

'2.17.0'

In [27]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

gpus = tf.config.list_physical_devices('GPU')

if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

Num GPUs Available:  0


In [14]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
import tensorflow as tf

train_dir = 'images'
val_dir = 'images'

def extract_class(filename):
    return 1 if 'day' in filename else 0

train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    directory=train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=True
)

val_generator = val_datagen.flow_from_directory(
    directory=val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

file_names = train_generator.filenames
labels = train_generator.classes

df = pd.DataFrame({
    'File Name': file_names,
    'Label': labels
})

print(df)
print(len(df))

# Define Focal Loss
def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1. - epsilon)
        y_true = K.cast(y_true, tf.float32)
        alpha_t = y_true * alpha + (1 - y_true) * (1 - alpha)
        p_t = y_true * y_pred + (1 - y_true) * (1 - y_pred)
        fl = - alpha_t * K.pow((1 - p_t), gamma) * K.log(p_t)
        return K.mean(fl)
    return focal_loss_fixed

# Load ResNet50 base model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=x)

# Compile with Focal Loss
model.compile(optimizer=Adam(), loss=focal_loss(gamma=2., alpha=0.25), metrics=['accuracy'])

# Train the model
model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

# Evaluate the model
val_loss, val_acc = model.evaluate(val_generator)
print(f"Validation accuracy: {val_acc}")


Found 6306 images belonging to 2 classes.
Found 6306 images belonging to 2 classes.
                                              File Name  Label
0                                   train/day_00010.jpg      0
1                                   train/day_00019.jpg      0
2                                   train/day_00026.jpg      0
3                                   train/day_00027.jpg      0
4                                   train/day_00029.jpg      0
...                                                 ...    ...
6301                              valid/night (990).jpg      1
6302                              valid/night (991).jpg      1
6303                              valid/night (992).jpg      1
6304                              valid/night (993).jpg      1
6305  valid/.ipynb_checkpoints/day_00004-checkpoint.jpg      1

[6306 rows x 2 columns]
6306


/home/amirhosein/.pyenv/versions/3.11.9/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 174s 871ms/step - accuracy: 0.6345 - loss: 0.1291 - val_accuracy: 0.6637 - val_loss: 0.0754
Epoch 2/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 175s 883ms/step - accuracy: 0.6649 - loss: 0.0682 - val_accuracy: 0.6637 - val_loss: 0.0869
Epoch 3/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 175s 884ms/step - accuracy: 0.6633 - loss: 0.0727 - val_accuracy: 0.6640 - val_loss: 0.0638
Epoch 4/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 178s 899ms/step - accuracy: 0.6569 - loss: 0.0665 - val_accuracy: 0.6638 - val_loss: 0.0632
Epoch 5/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 178s 901ms/step - accuracy: 0.6664 - loss: 0.0656 - val_accuracy: 0.6641 - val_loss: 0.0680
Epoch 6/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 181s 917ms/step - accuracy: 0.6708 - loss: 0.0656 - val_accuracy: 0.6637 - val_loss: 0.0629
Epoch 7/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 177s 897ms/step - accuracy: 0.6699 - loss: 0.0639 - val_accuracy: 0.6637 - val_loss: 0.0629
Epoch 8/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 176s 889ms/step - accuracy: 0.6764 -

In [3]:
model.save_weights('day_night_model.weights.h5')

In [12]:
def load_trained_model():
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    
    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=base_model.input, outputs=x)
    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

    model.load_weights('day_night_model.weights.h5')

    return model

def classify_image(model, img_path):
    if os.path.exists(img_path):
        print(f"Loading image from: {img_path}")

        print(img_path)
        img = image.load_img(img_path, target_size=(224, 224))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array)

        prediction = model.predict(img_array)
        
        print(f"Prediction value: {prediction}")

        return 'Day' if prediction > 0.5 else 'Night'
    else:
        print(f"File not found: {img_path}")
        return None

def classify_images_in_directory(model, directory):
    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)
        
        if os.path.isfile(filepath):
            true_label = extract_class(filename)
            
            predicted_label = classify_image(model, filepath)
            
            if predicted_label is not None:
                print(f"File: {filename}, True Label: {'Day' if true_label == 1 else 'Night'}, "
                      f"Predicted: {predicted_label}")
        else:
            print(f"Skipping {filepath}, not a file.")

In [13]:
if __name__ == "__main__":
    model = load_trained_model()

    test_images = [
        'images/train/day_00010.jpg',
        'images/train/day_00019.jpg',
        'images/train/day_00026.jpg',
        'images/train/day_01146.jpg',
        'images/train/day_01176.jpg',
        'images/train/day_01246.jpg',
        'images/train/day_01372.jpg',
        'images/train/night (3074).jpg',
        'images/train/night (3078).jpg',
        'images/train/night (3079).jpg',
        'images/train/night (3131).jpg',
        'images/train/night (3146).jpg',
        'images/train/night (3241).jpg',
        'images/train/night (3707).jpg',
    ]
    
    validation_images = [
        'images/valid/day_00018.jpg',
        'images/valid/day_01179.jpg',
        'images/valid/day_01274.jpg',
        'images/valid/day_01421.jpg',
        'images/valid/day_01614.jpg',
        'images/valid/day_02040.jpg',
        'images/valid/day_00281.jpg',
        'images/valid/night (1419).jpg',
        'images/valid/night (2302).jpg',
        'images/valid/night (2277).jpg',
        'images/valid/night (2040).jpg',
        'images/valid/night (1756).jpg',
        'images/valid/night (1435).jpg',
        'images/valid/night (1432).jpg',
    ]

    for img_path in test_images:
        result = classify_image(model, img_path)
        
        if result:
            print(f"The image '{img_path}' is classified as: {result}")
    
    for img_path in validation_images:
        result = classify_image(model, img_path)
        
        if result:
            print(f"The image '{img_path}' is classified as: {result}")


/home/amirhosein/.pyenv/versions/3.11.9/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Loading image from: images/train/day_00010.jpg
images/train/day_00010.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 621ms/step
Prediction value: [[0.9910168]]
The image 'images/train/day_00010.jpg' is classified as: Day
Loading image from: images/train/day_00019.jpg
images/train/day_00019.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Prediction value: [[0.9767325]]
The image 'images/train/day_00019.jpg' is classified as: Day
Loading image from: images/train/day_00026.jpg
images/train/day_00026.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Prediction value: [[0.9372569]]
The image 'images/train/day_00026.jpg' is classified as: Day
Loading image from: images/train/day_01146.jpg
images/train/day_01146.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Prediction value: [[0.99101126]]
The image 'images/train/day_01146.jpg' is classified as: Day
Loading image from: images/train/day_01176.jpg
images/train/day_01176.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
Prediction value: [[0.9983656]]
The image 'images/train/day_01176.